- 1. 검색어 하고 재생수 기준으로 네이버 tv에서 띄움.
- 2. item이 없는 페이지 -1까지의 페이지 index를 저장한다.
- 3. 저장된 페이지 index 범위 내에서 작업한다.
- 3. tv프로그램 리스트에 있는 채널들의 동영상의 (프로그램 명,제목,날짜,설명, 해시태그는 나중에) 정보를 가져온다. 
- 4. 가져온 데이터를 날짜 순으로 정렬한 뒤 엑셀로 저장 

이 과정만 파이썬 코드로 


# 네이버 TV 크롤링

## 1.1 사용 패키지

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

## 1.2 이용방법

- 서버에서 Naver_Blog_Crawler/TV_Crawler.ipynb 실행

- 키워드 INPUT : ../Naver_Blog_Crawler/name.txt

- OUTPUT (CSV) : Naver_Blog_Crawler/data/"keyword".txt

## 1.3 출력물

#### 1) 변수 설명

    - program : 해당 영상의 프로그램 이름
    - title : 해당 영상 제목
    - hit : 해당 영상 조회수
    - date : 해당 영상 게시 날짜
    - url : 해당 영상 url 주소

## 1.4 코드

### - 입력란

In [32]:
keyword_df=pd.read_csv("네이버TV 키워드.csv",encoding='ms949')
keywordlist=keyword_df['품명'].values.tolist()

### - 코드 실행

#### 각 키워드에 대한 자료가 몇 페이지까지 있는지를 확인한다.

In [34]:
tv_pro_pd=pd.read_csv("TVprogram.csv",encoding='ms949')
tv_pro_list=tv_pro_pd['프로그램명'].values.tolist()

index_list = {}
#각 키워드에 대한 반복문 실행
for keyword in keywordlist:

    page_index=1
    stop_index=0
    while True:
        response=requests.get("https://tv.naver.com/search/clip?query="+keyword+"&sort=date&page="+str(page_index)+"&isTag=false")
        html=response.text
        soup=BeautifulSoup(html,'html.parser')  
        tag=soup.find("div",{"class":"src_wrap"}).text
        tag_text_list=tag.split("\n")
        #print(page_index)
        #네이버 TV는 자료가 있는 만큼 페이지 수를 제공하는 게 아니라 계속 증가한다.
        #다만 자료가 없을 경우는 아래의 문장을 출력하기 때문에 그것으로 자료를 다 끌어왔음을 확인한다.
        if '- 단어의 철자가 정확한지 확인해주세요.' in tag_text_list :
            stop_index=page_index-1
            break
        page_index+=1
    index_list[keyword] = stop_index
    print(keyword+" "+str(stop_index))

바지락 32


#### 각 키워드에 대한 페이지 수가 정해지면 그만큼만 크롤링을 진행한다. 

In [35]:
for keyword in keywordlist:
    stop_index = index_list[keyword]
    print(keyword)
    program_list=[]
    title_list=[]
    hit_list=[]
    date_list=[]
    url_list=[]
    for page_index in range(1,stop_index):
        response=requests.get("https://tv.naver.com/search/clip?query="+keyword+"&sort=date&page="+str(page_index)+"&isTag=false")
        html=response.text
        soup=BeautifulSoup(html,'html.parser')
        tag_list=soup.find_all("div",{"class":"thl_a"})
        for tag in tag_list :
            publisher = tag.find("span",{"class":"ch_txt"}).find("a")
            if publisher.text in tv_pro_list:
                url_post="http://tv.naver.com"+tag.find("a",href=True)['href']
                url_post1=url_post.split("?")[0]
                response_post=requests.get(url_post)
                html_post=response_post.text
                soup_post=BeautifulSoup(html_post,'html.parser')
    
                title=soup_post.find("h3",{"class":"_clipTitle"}).text
                date=soup_post.find("span",{"class":"date"}).text
                hit=soup_post.find("span",{"class":"play"}).text
            
                #soup_hash=soup_post.find("ul",{"class":"list_hash"})
                #one_post_hash=soup_hash.text.split('\n')[1:-1]
                #print(soup_hash)
            
                date1=date.replace("등록","")
                hit1=hit.replace("        ","").replace("\n","").replace("재생수","")

                title_list.append(title)
                #print(title)
                date_list.append(date1)
                hit_list.append(hit1)
                #hash_list.append(one_post_hash)
                program_list.append(publisher.text)
                url_list.append(url_post1)
                
                
    finish=pd.DataFrame([program_list,title_list,hit_list,date_list,url_list]).T
    finish.columns=['프로그램','제목','조회수','날짜','URL']
    finish1=finish.drop_duplicates()
    finish1.sort_values(by='날짜').to_csv(keyword+"_네이버TV.csv",encoding='utf-8',index=False)   

바지락


In [ ]:
import os
os.getcwd()